In [17]:
import tensorflow_datasets as tf_ds
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np


## Data preparation

In [32]:

# download the data
data = tf_ds.load('cifar10_1/v6', split='test')

In [34]:
data

<PrefetchDataset element_spec={'image': TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [41]:
tensor_dict = next(iter(data))
print('Shape of image', tensor_dict['image'].shape)
print('Shape of label', tensor_dict['label'].shape)

Shape of image (32, 32, 3)
Shape of label ()


In [43]:
images, labels = [], []

for example in data:
    images.append(example['image'].numpy())
    labels.append(example['label'].numpy())
        

In [44]:
images = np.array(images)
labels = np.array(labels)

print('shape of images',images.shape)
print('shape of labels',labels.shape)

shape of images (2000, 32, 32, 3)
shape of labels (2000,)


In [47]:
# train test split

train_images = images[:1500]
test_images = images[1500:]
train_labels = labels[:1500]
test_labels = labels[1500:]


In [50]:
test_images.reshape(test_images.shape[0],-1).shape

(500, 3072)

## Nearest Neighbours

In [61]:
%%writefile './Nearest_Neighbours.py'

import numpy as np

class NearestNeighbours:

    def __init__(self, images, labels):
        self.train_images = images.reshape(images.shape[0],-1)
        self.train_labels = labels

    def __call__(self, images):
        num_images = images.shape[0]
        test_images = images.reshape(num_images,-1)
        preds = np.zeros(num_images, dtype=np.int32)

        # find nearest training image for each of the test-images
        # using L1-distance (sum of absolute differences)
        for i in range(num_images):
            L1 = np.sum(np.abs(self.train_images - test_images[i,:]), axis=1)
            least_dist_id = np.argmin(L1)
            preds[i]=self.train_labels[least_dist_id]
        
        return preds

Overwriting ./Nearest_Neighbours.py


In [53]:
# create an instance
neighbour = NearestNeighbours(train_images, train_labels)
predictions = neighbour(test_images)

In [64]:
accuracy = np.sum(predictions  == test_labels) / len(predictions)
print('Model accuracy',accuracy)

Model accuracy 0.168


In [65]:
# correct prediction (true positives)
t = np.sum(predictions==test_labels)
f = len(predictions) - t

print('Correct prediction ',t)
print('Incorrect prediction ',f)

Correct prediction  84
Incorrect prediction  416
